In [ ]:
import pandas as pd
import os
from PIL import Image, ImageDraw
import random
import math
from tqdm import tqdm

In [ ]:


def draw_random_elliptical_blob(draw, center, avg_radius_x, avg_radius_y, irregularity, spikiness, num_points, color):
    cx, cy = center
    angle = 0
    points = []
    for i in range(num_points):
        angle_step = 2 * math.pi / num_points
        # Reduce angular irregularity for more even spacing
        angle += angle_step + random.uniform(-angle_step * irregularity, angle_step * irregularity)
        # For vertical radius, reduce spikiness to avoid thin tops/bottoms
        radius_x = avg_radius_x + random.uniform(-avg_radius_x * spikiness, avg_radius_x * spikiness)
        radius_y = avg_radius_y + random.uniform(-avg_radius_y * (spikiness * 0.4), avg_radius_y * (spikiness * 0.4))
        x = cx + math.cos(angle) * radius_x
        y = cy + math.sin(angle) * radius_y
        points.append((x, y))
    draw.polygon(points, fill=color)

def blobs_on_image(input_path, output_path, color, num_blobs=7):
    image = Image.open(input_path).convert("RGBA")
    draw = ImageDraw.Draw(image, 'RGBA')
    w, h = image.size
    for _ in range(num_blobs):
        center = (random.randint(0, w), random.randint(0, h))
        avg_radius_x = random.randint(30, 55)            # Moderate horizontal radius
        avg_radius_y = random.randint(80, 150)           # Taller vertical radius
        irregularity = random.uniform(0.15, 0.3)          # Moderate angular variation
        spikiness = random.uniform(0.2, 0.35)             # Moderate radius variation
        num_points = random.randint(20, 30)               # More points for smooth, round shapes
        draw_random_elliptical_blob(draw, center, avg_radius_x, avg_radius_y,
                                    irregularity, spikiness, num_points, color)
    image.save(output_path, "PNG")




In [ ]:
def batch_process_images():
    test_df = pd.read_pickle("../data/pepper_data_test_equal.pkl")
    image_paths = test_df['image_path_env'].tolist()
    output_dir = "../data/perturbations/environment"
    os.makedirs(output_dir, exist_ok=True)

    imagenet_mean = (0.485, 0.456, 0.406)
    blob_color = tuple(int(x * 255) for x in imagenet_mean) + (255,)

    for img_path in tqdm(image_paths):
        # Extract file name and compose output path
        file_name = os.path.basename(img_path)
        output_path = os.path.join(output_dir, file_name)
        blobs_on_image(
            input_path=img_path,
            output_path=output_path,
            color=blob_color,
            num_blobs=25
        )

# if __name__ == "__main__":
batch_process_images()